In [1]:
import os

In [2]:
pwd

'/Users/shashank/Documents/Python_Stuff/End-to-end-project-implementation/research'

In [3]:
#  changing the directory using os library
os.chdir("../")

In [4]:
pwd

'/Users/shashank/Documents/Python_Stuff/End-to-end-project-implementation'

In [5]:
# We use dataclasses as decorator to define user defined return type.
from dataclasses import dataclass
from pathlib import Path

# this dataclass will be used for accesing config that we defined.
@dataclass (frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
# Config>> configuration
# Defining the configuration here for testing purpose only.
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [13]:
import os
import urllib
import requests
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [14]:
#  Components> data ingestion
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            print(f"URL of data: {self.config.source_URL}")
            # filename, headers = request.urlretrieve(
            #     url = self.config.source_URL,
            #     filename = self.config.local_data_file
            # )
            headers = self.download_url(self.config.source_URL,self.config.local_data_file)
            logger.info(f" download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    # def download_url(self,url, save_path):
    #     with urllib.request.urlopen(url) as dl_file:
    #         with open(save_path, 'wb') as out_file:
    #             out_file.write(dl_file.read())

    def download_url(url, save_path, chunk_size=128):
        r = requests.get(url, stream=True)
        with open(save_path, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=chunk_size):
                fd.write(chunk)

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
# stage_1 Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-08 07:26:35,657: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-08 07:26:35,660: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-08 07:26:35,661: INFO: common: created directory at: artifacts]
[2023-12-08 07:26:35,661: INFO: common: created directory at: artifacts/data_ingestion]
URL of data: https://github.com/theshashanksinghal/CNN-from-scratch-and-One-shot-learning/raw/master/data/data.zip


MissingSchema: Invalid URL '<__main__.DataIngestion object at 0x110120cd0>': No scheme supplied. Perhaps you meant https://<__main__.DataIngestion object at 0x110120cd0>?